# DOE WPTO Wave Hindcast Data Access

This notebook provides examples on accessing the 32-year Wave Hindcast (1979-2010) dataset with brief examples on using the data within MHKiT and the SAMs tool. This data was generated via a collaboration between Pacific Northwest National Lab, Sandia National Lab, and the National Renewable Energy Lab and is hosted from Amazon Web Services using the HDF Group's Highly Scalable Data Service (HSDS) for public access.

Currently the dataset only includes the U.S. West Coast, but it will grow incrementally over the next few years to include all U.S. regions by 2022. The dataset will also be extended to span 1979-2020 (42 years) by late 2022.

## Dataset Description

There are two major sub-datasets within the overarching dataset:

1. The high-spatial-resolution dataset (hereafter the 'spatial dataset/data') spans the U.S. Exclusive Economic Zone (EEZ) with an unstructured grid that has ~200 m resolution in shallow water. The time step resolution for this spatial data is 3-hours and spans 32 years, 01/01/1979 - 12/31/2010.
  - The spatial dataset is organized into yearly files (each file contains all variables and locations in that year) and they are located on AWS at: `/nrel/US_wave/US_wave_{year}.h5`
  - This dataset includes the following variables (indexed by 'coordinates' (latitude and longitude), and a 'time_index'):
    - directionality_coefficient (no units)
    - energy_period (seconds)
    - maximum_energy_direction (degrees true, direction from-which waves are travelling)
    - mean_absolute_period (seconds)
    - mean_zero-crossing_period (seconds)
    - omni-directional_wave_power (Watts)
    - peak_period (seconds)
    - significant_wave_height (meters)
    - spectral_width (no units)
    - water_depth (meters)

2. The 'virtual buoy dataset' is also available at specific locations within the large spatial domain. These virtual buoys span the same 32-years of the spatial dataset however the time resolution is 1-hour, and these data also include the wave directional spectrum.
  - The virtual buoy dataset is located on AWS at: `/nrel/US_wave/virtual_buoy/US_virtual_buoy_{year}.h5`
  - This dataset includes all of the variables listed above, and also includes the wave spectrum (indexed by 'time_index','frequency', 'direction', and 'coordinates'):
    - direction_wave_spectrum (meters<sup>2</sup>/(degree*Hz)

## Installing and Configuring the HSDS service

For this example to work, the packages necessary can be installed via pip:

```
$ pip install -r requirements.txt
```

Next you'll need to configure h5pyd to access the data on HSDS:

```
$ hsconfigure
```

and enter at the prompt:

```
hs_endpoint = https://developer.nrel.gov/api/hsds   
hs_username = None   
hs_password = None   
hs_api_key = 3K3JQbjZmWctY0xmIfSYvYgtIcM3CN0cb1Y2w9bf    
```

The example API key here is for demonstation and is rate-limited per IP. To get your own API key, visit https://developer.nrel.gov/signup/. Alternatively, you can add the above contents to the hsds configuration file: `~/.hscfg`

Use [Jupyter Notebook or Jupyter Lab](https://jupyter.org) to view and modify this example notebook. For example, after following the steps above, start a jupyter notebook by:
```
$ jupyter notebook
```

# Basic Usage of the Spatial Dataset

The following examples illustrate basic examples using NREL-rex to access and download parts of the WPTO Wave Hindcast dataset.

Datasets are returned as Pandas objects. See the Pandas [documentation](https://pandas.pydata.org/pandas-docs/stable/) 
for more information about working with Pandas objects.

We start by viewing the 'index variables' of the dataset.

In [ ]:
# quick View of metadata, time_index, and coordinates 
from rex import WaveX

waveFile = f'/nrel/US_wave/US_wave_1990.h5'

with WaveX(waveFile, hsds=True) as waves:
    meta = waves.meta          ## meta is an object that contains location information for each data point
    print("meta =", meta)
    time_index = waves.time_index # time_index contains all of the years within the file
    print("time_index =",time_index)
    coordinates = waves.coordinates # coordinates contains all the lat/lon pars within the dataset
    print("coordinates =",coordinates)
    

## Extracting data from a single site:
A single lat/lon pair can be given to extract data nearest to that location. 

In [ ]:
# Extract the timeseries of significant_wave_height closest to a coordinate pair
from rex import WaveX

with WaveX(waveFile, hsds=True) as waves:
    lat_lon = (44.624076,-124.280097)
    parameters = 'significant_wave_height'
    swh_single = waves.get_lat_lon_df(parameters, lat_lon)
swh_single

## Extracting data from multiple sites:
A list of latitude/longitude pairs can be passed to extract data from multiple sites

In [ ]:
# Extract the timeseries of significant_wave_height closest to multiple coordinate pairs
from rex import WaveX

with WaveX(waveFile, hsds=True) as waves:
    lat_lon = [(44.624076,-124.280097),(43.489171,-125.152137)] # set lat_lon to a list of lat/lon pairs
    parameters = 'significant_wave_height'
    swh_multi = waves.get_lat_lon_df(parameters, lat_lon)
swh_multi

## Extracting Data over Multiple Years:
The examples above return data for a single year only. To extract data over a number of years, use the `MultiYearWaveX` class:

In [ ]:
# Concatonate yearly significant wave height datasets into a single timeseries
from rex import MultiYearWaveX # Yearly concatonation requires the MultiYearResource function

multi_year_waves = f'/nrel/US_wave/US_wave_199*.h5' # Use wildcards to indicate what years to include.

with MultiYearWaveX(multi_year_waves,hsds=True) as mYears:
    lat_lon = (44.624076,-124.280097)
    parameters = 'significant_wave_height'
    swh_multi_year = mYears.get_lat_lon_df(parameters, lat_lon)
    
swh_multi_year
    

This functionality extends to multiple locations and regions

In [ ]:
# Concatonate yearly significant wave height datasets for multiple llocations into a single timeseries
from rex import MultiYearWaveX 

multi_year_waves = f'/nrel/US_wave/US_wave_199*.h5' 

with MultiYearWaveX(multi_year_waves, hsds=True) as waves:
    lat_lon = [(44.624076,-124.280097),(43.489171,-125.152137)] # set lat_lon to a list of lat/lon pairs
    parameters = 'significant_wave_height'
    swh_multi = waves.get_lat_lon_df(parameters, lat_lon)

swh_multi

# Basic Usage of the Virtual Buoys

Virtual buoys were output from theUnSWAN model runs and contain 1-hour timestep data with directional spectrum data

Virtual Buoy dataset fill names follow the convention: "/nrel/US_wave/virtual_buoy/US_virtual_buoy_{year}.h5"

These data can be accessed using the same basic approaches described above, but using a different path that points to these data: `/nrel/US_wave/virtual_buoy/US_virtual_buoy_{year}.h5`

Two examples are below, the first accessing a single year of data:

In [ ]:
# View the metadata and time_index for the virtual buoy datasets
from rex import WaveX

vBuoy = f'/nrel/US_wave/virtual_buoy/US_virtual_buoy_1995.h5'

with WaveX(vBuoy, hsds=True) as waves:
    meta = waves.meta          ## meta is an object that contains location information for each data point
    print("meta =", meta)
    time_index = waves.time_index # time_index contains all of the years within the file
    print("time_index =",time_index)
    

The second uses `MultiYearWaveX` to get multiple years of data:

In [ ]:
# Download and concatonate multiple years of significant wave height data from teh virtual buoy dataset
from rex import MultiYearWaveX

multi_year_vBuoy = f'/nrel/US_wave/virtual_buoy/US_virtual_buoy_199*.h5' # Grab all data from the 1990s

with MultiYearWaveX(multi_year_vBuoy,hsds=True) as mYears:
    lat_lon = (44.624076,-124.280097)
    parameters = 'significant_wave_height'
    swh_buoy = mYears.get_lat_lon_df(parameters, lat_lon)
    
swh_buoy

# Integration with MHKiT

Functionality to read the WPTO hindcast data is currently being integrated into [MHKiT](https://mhkit-software.github.io/MHKiT/), and will be avaiable to users soon. The following examples show how you will be able to use MHKiT to access the dataset once the functions are integrated into MHKiT. In addition to the Python functions demonstrated below, MHKiT-MATLAB functions will also be available with the same functionality.  

Note: The following examples currently will NOT run on your local machine. 

In [ ]:
# Importing local development version of MHKiT. This code will NOT work on local machines. 
import sys
sys.path.insert(1, '/mnt/c/Users/abharath/Documents/Projects/MHKit/')

### Extracting data from a single site:
A single lat/lon pair can be given to extract data nearest to that location. 

In [ ]:
# Extract the timeseries of significant_wave_height closest to a coordinate pair
from mhkit.wave import io

single_year_waves = f'/nrel/US_wave/US_wave_1995.h5'
lat_lon = (44.624076,-124.280097)
parameters = 'significant_wave_height'

wave_singleYear = io.read_US_wave_dataset(single_year_waves,parameters,lat_lon)

wave_singleYear

### Extracting data from multiple sites:
A list of latitude/longitude pairs can be passed to extract data from multiple sites

In [ ]:
# Extract the timeseries of significant_wave_height closest to multiple coordinate pairs
from mhkit.wave import io

single_year_waves = f'/nrel/US_wave/US_wave_1995.h5'
lat_lon = [(44.624076,-124.280097),(43.489171,-125.152137)]
parameters = 'significant_wave_height'

wave_multiLocal = io.read_US_wave_dataset(single_year_waves,parameters,lat_lon)

wave_multiLocal

### Extracting Data over Multiple Years:
Data can be extracted over a number of years and concatonated directly with the MHKiT tools.
The new multi-year object has the same functionality as a single year dataset. The datasets are concatonated into a single timeseries for convenience 

In [ ]:
# Extract and concatonate a multi-year the timeseries of significant_wave_height closest to a coordinate pair
from mhkit.wave import io

multi_year_waves = f'/nrel/US_wave/US_wave_199*.h5'
lat_lon = (44.624076,-124.280097)
parameters = 'significant_wave_height'

wave_multiYear = io.read_US_wave_dataset(multi_year_waves,parameters,lat_lon)

wave_multiYear

In a similar way to the pervious virtual buoy examples using the rex package, Virtual buoy data can be accessed through MHKiT-Python simply by selecting the correct file location

# Working with Dataset Bulk Parameters

In the following example we provide a simple means to view the data from the WPTO Wave Hindcast Datasets

In [ ]:
# pull datasets from AWS
from mhkit.wave import io

dataset = f'/nrel/US_wave/US_wave_1995.h5'
lat_lon = (44.624076,-124.280097)
swh_name, owp_name = 'significant_wave_height', 'omni-directional_wave_power'

# First we use the MHKiT loading function to grab the datasets from AWS 
swh = io.read_US_wave_dataset(dataset,swh_name,lat_lon)
owp = io.read_US_wave_dataset(dataset,owp_name,lat_lon)

In [ ]:
%config InlineBackend.figure_format ='retina'
%matplotlib widget

from mpl_toolkits.axes_grid1 import host_subplot
import matplotlib.pyplot as plt

host = host_subplot(111)
par = host.twinx()

host.set_xlabel("Time")
host.set_ylabel(swh_name)
par.set_ylabel(owp_name)

p1, = host.plot(swh.index, swh.values, label=swh_name)
p2, = par.plot(owp.index, owp.values, label=owp_name)

leg = plt.legend()

host.yaxis.get_label().set_color(p1.get_color())
leg.texts[0].set_color(p1.get_color())

par.yaxis.get_label().set_color(p2.get_color())
leg.texts[1].set_color(p2.get_color())

plt.show()

# Accessing and Working with the Virtual Buoy Direction Wave Spectum

## Capture Length and Power Matrices Calculations with MHKiT

The WPTO hindcast data and MHKiT can be used to compute capture length and power matrices. The following example demonstrates this workflow using a synthetically generated power dataset. 

In [ ]:
from pandas import Series
from numpy.random import seed, normal

owp = 'omni-directional_wave_power'
J = io.read_US_wave_dataset(single_year_waves,owp,lat_lon)

# Set the random seed, to reproduce results
seed(1)                                               
# Generate random power values
P = Series(normal(200, 40, J.shape[0]),index = J.index)

In [ ]:
from mhkit.wave.performance import wave_energy_flux_matrix, capture_length, capture_length_matrix, power_matrix
from numpy import arange

Te = io.read_US_wave_dataset(single_year_waves,'energy_period',lat_lon)
Hm0 = io.read_US_wave_dataset(single_year_waves,'significant_wave_height',lat_lon)

# Calculate capture length
L = capture_length(P, J) 

# Generate bins for Hm0 and Te, input format (start, stop, step_size)
Hm0_bins = arange(0, Hm0.values.max() + .5, .5)    
Te_bins = arange(0, Te.values.max() + 1, 1)

# Create capture length matrix using a mean
LM = capture_length_matrix(Hm0, Te, L, 'mean', Hm0_bins, Te_bins)
# Create wave energy flux matrix using mean
JM = wave_energy_flux_matrix(Hm0, Te, J, 'mean', Hm0_bins, Te_bins)

# Create power matrix using mean
PM = power_matrix(LM, JM)

In [ ]:
from mhkit.wave.graphics import plot_matrix
# Plot the Plot mean matrix
ax = plot_matrix(PM)
